In [29]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')

from model import ode_y_damage_ambiguity, uncertainty_decomposition_damage_ambiguity
from simulation import simulation
from utilities import find_nearest_value
from numba import njit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [33]:
ξ_a = .01
# ξ_a = 1000.

η = .032
δ = .01

θ = pd.read_csv('../data/model144.csv', header=None).to_numpy()[:, 0]/1000.
πc_o = np.ones_like(θ)/len(θ)
σ_y = 1.2*np.mean(θ)

y_bar = 2.
γ_1 = 1.7675/10000
γ_2 = .0022*2
# γ_2p = np.array([0, .0197*2, .3853*2])
γ_2p = np.array([0, .0197*2])
# γ_2p = np.array([0, .0197*2*2, .3853*2])

y_step = .02
# y_grid = np.arange(0., 2.+y_step, y_step)
y_grid = np.arange(0., 4.+y_step, y_step)

n_damage = len(γ_2p)
n_temp = 16
n_carb = 9

# Combine θ and γ_2p
θ_new = np.zeros((len(γ_2p), len(θ)))
γ_2p_new = np.zeros((len(γ_2p), len(θ)))
for i in range(len(γ_2p)):
    for j in range(len(θ)):
        θ_new[i, j] = θ[j]
        γ_2p_new[i, j] = γ_2p[i]
θ_new = θ_new.reshape(-1)
γ_2p_new = γ_2p_new.reshape(-1)
πc_o_new = np.ones_like(θ_new)/len(θ_new)

model_paras = (η, δ, θ_new, πc_o_new, σ_y, ξ_a, γ_1, γ_2, γ_2p_new, y_bar) 
model_res = ode_y_damage_ambiguity(y_grid, model_paras, v0=None, ϵ=1.,
                  tol=1e-8, max_iter=5_000, print_all=False)

Converged. Total iteration 1489: LHS Error: 9.922130672634921e-09; RHS Error 0.0004074733597089694


In [34]:
# model_res['e_tilde'] = np.ones_like(model_res['e_tilde'])*10
# model_res['e_tilde'] = np.load('e_star.npy')
np.save('emission_damage_ambiguity', model_res['e_tilde'])

In [35]:
n_damage = len(γ_2p)
n_temp = 16
n_carb = 9

θ_reshape = θ.reshape(n_temp, n_carb)
θ_all = np.zeros((n_damage, n_temp, n_carb))
γ_2p_all = np.zeros((n_damage, n_temp, n_carb))

for i in range(n_damage):
    for j in range(n_temp):
        for n in range(n_carb):
            θ_all[i, j, n] = θ_reshape[j, n]
            γ_2p_all[i, j, n] = γ_2p[i]

θ_damage = θ_all.mean(axis=(1, 2))
θ_temp = θ_all.mean(axis=(0, 2))
θ_carb = θ_all.mean(axis=(0, 1))
γ_2p_damage = γ_2p_all.mean(axis=(1, 2))
γ_2p_temp = γ_2p_all.mean(axis=(0, 2))
γ_2p_carb = γ_2p_all.mean(axis=(0, 1))
πc_o_damage = np.ones_like(θ_damage)/len(θ_damage)
πc_o_temp = np.ones_like(θ_temp)/len(θ_temp)
πc_o_carb = np.ones_like(θ_carb)/len(θ_carb)

θ_damage_temp = θ_all.mean(axis=2).reshape(-1)
θ_damage_carb = θ_all.mean(axis=1).reshape(-1)
θ_temp_carb = θ_all.mean(axis=0).reshape(-1)
γ_2p_damage_temp = γ_2p_all.mean(axis=2).reshape(-1)
γ_2p_damage_carb = γ_2p_all.mean(axis=1).reshape(-1)
γ_2p_temp_carb = γ_2p_all.mean(axis=0).reshape(-1)
πc_o_damage_temp = np.ones_like(θ_damage_temp)/len(θ_damage_temp)
πc_o_damage_carb = np.ones_like(θ_damage_carb)/len(θ_damage_carb)
πc_o_temp_carb = np.ones_like(θ_temp_carb)/len(θ_temp_carb)

θ_baseline = θ_all.mean().reshape(-1)
γ_2p_baseline = γ_2p_all.mean().reshape(-1)
πc_o_baseline = np.ones_like(θ_baseline)/len(θ_baseline)

θ_total = θ_all.reshape(-1)
γ_2p_total = γ_2p_all.reshape(-1)
πc_o_total = np.ones_like(θ_total)/len(θ_total)

# Baseline
model_paras_baseline = (η, δ, θ_baseline, πc_o_baseline, σ_y, ξ_a, γ_1, γ_2, γ_2p_baseline, y_bar) 
model_baseline = uncertainty_decomposition_damage_ambiguity(y_grid, model_res['e_tilde'], model_paras_baseline, v0=None, ϵ=1.,
                  tol=1e-8, max_iter=5_000, print_all=False)

# Damage uncertainty
model_paras_damage = (η, δ, θ_damage, πc_o_damage, σ_y, ξ_a, γ_1, γ_2, γ_2p_damage, y_bar) 
model_damage = uncertainty_decomposition_damage_ambiguity(y_grid, model_res['e_tilde'], model_paras_damage, v0=None, ϵ=1.,
                  tol=1e-8, max_iter=5_000, print_all=False)

# Temperature uncertainty
model_paras_temp = (η, δ, θ_temp, πc_o_temp, σ_y, ξ_a, γ_1, γ_2, γ_2p_temp, y_bar) 
model_temp = uncertainty_decomposition_damage_ambiguity(y_grid, model_res['e_tilde'], model_paras_temp, v0=None, ϵ=1.,
                  tol=1e-8, max_iter=5_000, print_all=False)

# Carbon uncertainty
model_paras_carb = (η, δ, θ_carb, πc_o_carb, σ_y, ξ_a, γ_1, γ_2, γ_2p_carb, y_bar) 
model_carb = uncertainty_decomposition_damage_ambiguity(y_grid, model_res['e_tilde'], model_paras_carb, v0=None, ϵ=1.,
                  tol=1e-8, max_iter=5_000, print_all=False)

# Damage + Temperature uncertainty
model_paras_damage_temp = (η, δ, θ_damage_temp, πc_o_damage_temp, σ_y, ξ_a, γ_1, γ_2, γ_2p_damage_temp, y_bar) 
model_damage_temp = uncertainty_decomposition_damage_ambiguity(y_grid, model_res['e_tilde'], model_paras_damage_temp, v0=None, ϵ=1.,
                  tol=1e-8, max_iter=5_000, print_all=False)

# Damage + Carbon uncertainty
model_paras_damage_carb = (η, δ, θ_damage_carb, πc_o_damage_carb, σ_y, ξ_a, γ_1, γ_2, γ_2p_damage_carb, y_bar) 
model_damage_carb = uncertainty_decomposition_damage_ambiguity(y_grid, model_res['e_tilde'], model_paras_damage_carb, v0=None, ϵ=1.,
                  tol=1e-8, max_iter=5_000, print_all=False)

# Temperature + Carbon uncertainty
model_paras_temp_carb = (η, δ, θ_temp_carb, πc_o_temp_carb, σ_y, ξ_a, γ_1, γ_2, γ_2p_temp_carb, y_bar) 
model_temp_carb = uncertainty_decomposition_damage_ambiguity(y_grid, model_res['e_tilde'], model_paras_temp_carb, v0=None, ϵ=1.,
                  tol=1e-8, max_iter=5_000, print_all=False)

# Damage + Temperature + Carbon uncertainty
model_paras_total = (η, δ, θ_total, πc_o_total, σ_y, ξ_a, γ_1, γ_2, γ_2p_total, y_bar) 
model_total = uncertainty_decomposition_damage_ambiguity(y_grid, model_res['e_tilde'], model_paras_total, v0=None, ϵ=1.,
                  tol=1e-8, max_iter=5_000, print_all=False)

Converged. Total iteration 1443: LHS Error: 9.961921954015907e-09; RHS Error 0.00029365657173054116
Converged. Total iteration 1477: LHS Error: 9.960514191220682e-09; RHS Error 0.0003540801534158561
Converged. Total iteration 1449: LHS Error: 9.914193022098061e-09; RHS Error 0.0003148543110500293
Converged. Total iteration 1445: LHS Error: 9.960102298478546e-09; RHS Error 0.0003016117209525965
Converged. Total iteration 1486: LHS Error: 9.966176550690875e-09; RHS Error 0.0003892692380071186
Converged. Total iteration 1481: LHS Error: 9.909224107929049e-09; RHS Error 0.00036723740427852675
Converged. Total iteration 1451: LHS Error: 9.910763765219599e-09; RHS Error 0.0003256667411466116
Converged. Total iteration 1490: LHS Error: 9.940414269493658e-09; RHS Error 0.0004074733607604894


In [36]:
np.save('ME_baseline_damage_ambiguity', model_baseline['ME'])
np.save('ME_damage_damage_ambiguity', model_damage['ME'])
np.save('ME_temp_damage_ambiguity', model_temp['ME'])
np.save('ME_carb_damage_ambiguity', model_carb['ME'])
np.save('ME_damage_temp_damage_ambiguity', model_damage_temp['ME'])
np.save('ME_damage_carb_damage_ambiguity', model_damage_carb['ME'])
np.save('ME_temp_carb_damage_ambiguity', model_temp_carb['ME'])
np.save('ME_damage_ambiguity', model_total['ME'])

array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
       0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
       0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
       0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
       0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  , 1.02, 1.04, 1.06, 1.08,
       1.1 , 1.12, 1.14, 1.16, 1.18, 1.2 , 1.22, 1.24, 1.26, 1.28, 1.3 ,
       1.32, 1.34, 1.36, 1.38, 1.4 , 1.42, 1.44, 1.46, 1.48, 1.5 , 1.52,
       1.54, 1.56, 1.58, 1.6 , 1.62, 1.64, 1.66, 1.68, 1.7 , 1.72, 1.74,
       1.76, 1.78, 1.8 , 1.82, 1.84, 1.86, 1.88, 1.9 , 1.92, 1.94, 1.96,
       1.98, 2.  , 2.02, 2.04, 2.06, 2.08, 2.1 , 2.12, 2.14, 2.16, 2.18,
       2.2 , 2.22, 2.24, 2.26, 2.28, 2.3 , 2.32, 2.34, 2.36, 2.38, 2.4 ,
       2.42, 2.44, 2.46, 2.48, 2.5 , 2.52, 2.54, 2.56, 2.58, 2.6 , 2.62,
       2.64, 2.66, 2.68, 2.7 , 2.72, 2.74, 2.76, 2.78, 2.8 , 2.82, 2.84,
       2.86, 2.88, 2.9 , 2.92, 2.94, 2.96, 2.98, 3.